In [1]:
import json
import gzip
import numpy as np
import pandas as pd
from pathlib import Path
from distutils.log import warn
from tqdm.notebook import tqdm

from pymatgen.core import Composition, Structure
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDEntry

tqdm.pandas()

In [2]:
import tensorflow as tf
import tensorflow_addons as tfa
import nfp
from nfp import custom_objects
from nfp.layers import RBFExpansion

from preprocess import preprocessor

In [3]:
# since package versions may be important
print(f"{pd.__version__ = }")
print(f"{np.__version__ = }")
print(f"{tf.__version__ = }")
print(f"{nfp.__version__ = }")

pd.__version__ = '1.4.2'
np.__version__ = '1.22.3'
tf.__version__ = '2.7.0'
nfp.__version__ = '0.3.12'


In [4]:
# Not sure which variable pymatgen uses for its version,
# so print it with pip
!pip show pymatgen

Name: pymatgen
Version: 2022.2.10
Summary: Python Materials Genomics is a robust materials analysis code that defines core object representations for structures and molecules with support for many electronic structure codes. It is currently the core analysis code powering the Materials Project (https://www.materialsproject.org).
Home-page: https://pymatgen.org
Author: Pymatgen Development Team
Author-email: ongsp@eng.ucsd.edu
License: MIT
Location: /home/jlaw/.conda-envs/crystals_nfp0_3/lib/python3.8/site-packages
Requires: matplotlib, monty, networkx, numpy, palettable, pandas, plotly, pybtex, requests, ruamel.yaml, scipy, spglib, sympy, tabulate, tqdm, uncertainties
Required-by: 


## Predict the total energy
- This model has been trained with the ICSD, fully relaxed, and volume-relaxed datasets
- This means it will predict the total energy of a structure in its given state
  - if the structure is unrelaxed, the energy predicted will be an upper-bound for the energy after relaxation

In [5]:
# Load the tensorflow model for predicting the total energy of a given structure
# If you haven't already, you'll need to download the model using download.sh (see README.md)
energy_model_file = Path("pretrained_models", "icsd_full_and_vol_battery.hdf5")
print(f"loading {energy_model_file}")
energy_model = tf.keras.models.load_model(energy_model_file,
                                          custom_objects={**custom_objects,
                                                          **{'RBFExpansion': RBFExpansion}})

loading pretrained_models/icsd_full_and_vol_battery.hdf5


Here's an example of how to predict the energy for a structure

In [6]:
# Load a structure from file. This is the LiSc2F7 relaxed structure from the paper
structure_file = "inputs/POSCAR_example"
structure = Structure.from_file(structure_file, primitive=True)
structure

Structure Summary
Lattice
    abc : 6.31324215781406 4.129266345748309 6.313262245757987
 angles : 89.99981298298509 115.93116321347914 90.00022269721498
 volume : 148.01081824725688
      A : 6.31320261273 -1.29824151174e-05 -0.0223453272811
      B : -7.5462453492e-06 4.12926634574 3.4182060174e-06
      C : -2.74061976372 1.08903819355e-05 5.6873793169
PeriodicSite: Li (3.1566, 3.8750, -0.0112) [0.5000, 0.9384, 0.0000]
PeriodicSite: Sc (3.2053, 3.8791, 3.9171) [0.8081, 0.9394, 0.6919]
PeriodicSite: Sc (0.3672, 3.8791, 1.7479) [0.1919, 0.9394, 0.3081]
PeriodicSite: F (1.0409, 3.8783, 3.8076) [0.4563, 0.9392, 0.6713]
PeriodicSite: F (2.5317, 3.8783, 1.8575) [0.5437, 0.9392, 0.3287]
PeriodicSite: F (5.1121, 3.8781, 0.2302) [0.8287, 0.9392, 0.0437]
PeriodicSite: F (-1.5396, 3.8782, 5.4349) [0.1713, 0.9392, 0.9563]
PeriodicSite: F (3.1538, 1.8145, 3.9495) [0.8024, 0.4394, 0.6976]
PeriodicSite: F (0.4188, 1.8145, 1.7155) [0.1976, 0.4394, 0.3024]
PeriodicSite: F (-1.3703, 3.8811, 2.8437) [

In [7]:
# Here we define some convenience functions for preprocessing structures and generating predictions
def preprocess_structure(structure):
    inputs = preprocessor(structure)
    # scale structures to a minimum of 1A interatomic distance
    min_distance = inputs["distance"].min()
    if np.isclose(min_distance, 0):
        warn(f"Error with {row.id}")
        return None

    scale_factor = 1.0 / inputs["distance"].min()
    inputs["distance"] *= scale_factor
    return inputs


def build_dataset(structures, batch_size=8):
    dataset = tf.data.Dataset.from_generator(
        lambda: (preprocess_structure(s) for s in structures),
        output_signature=(preprocessor.output_signature),
        ).padded_batch(
            batch_size=batch_size,
            padding_values=(preprocessor.padding_values),
        )
    return dataset


def predict_energy(energy_model, structures):
    dataset = build_dataset(structures)
    predicted_energy = energy_model.predict(dataset)
    return predicted_energy.flatten()

In [8]:
predicted_energies = predict_energy(energy_model, [structure])
predicted_energies

array([-6.284776], dtype=float32)

In [9]:
# The fully-relaxed energy for this structure is -6.302
energy_error = -6.302 - predicted_energies[0]
print(f"{abs(energy_error):0.4f}")

0.0172


A prediction error of 17 meV/atom is not bad!

## Decomposition Energy
- Build the convex hull and compute the predicted decomposition energy for this structure
- We can also compute all of the "sub-rewards" we're interested in
- To compute the self-consistent decomposition energy, we would need all the DFT-relaxed energies we have, as well as the lowest predicted energy for each relevant composition

In [10]:
from src.ehull import setup_competing_phases
from src.crystal_reward import StructureRewardBattInterface

In [11]:
# first load the competing phases from NREL MatDB
competing_phases_file = "inputs/competing_phases.csv"
competing_phases = setup_competing_phases(competing_phases_file)

Reading inputs/competing_phases.csv
	12682 lines
  sortedformula   icsdnum  energyperatom reduced_composition
0    Ag10Br3Te4  173116.0      -1.718985          Ag10Br3Te4
1   Ag11K1O16V4  391344.0      -4.797702         Ag11K1O16V4
	12682 entries


In [12]:
# The rewarder calculates all the sub-rewards (e.g. decomposition energy, conducting ion %)
# and combines them into a single reward value
rewarder = StructureRewardBattInterface(competing_phases)

In [13]:
# now compute the reward of the structure
comp = structure.composition
energyperatom = predicted_energies[0]
reward, info = rewarder.compute_reward(comp, energyperatom)
comp, reward, info

(Comp: Li1 Sc2 F7,
 0.8275238356399739,
 {'predicted_energy': -6.284776,
  'oxidation': -5,
  'reduction': -0.8864,
  'stability_window': 4.1136,
  'decomp_energy': -1.7769,
  'cond_ion_frac': 0.1})

## Model Evaluation
- `train_model.py` already predicts the energy of the test set after training 
- Here we can print the Mean Absolute Error (MAE) for each dataset
- See `pretrained_models/plot_err.ipynb` for more

In [14]:
run_dir = "pretrained_models"
df_pred = pd.read_csv(Path(run_dir, "predicted_energies.csv.gz"),
                      index_col=0)
df_pred.head(2)

,id,composition,energyperatom,volume,num_sites,dataset,scale_factor,set,energy_predicted
0,icsd_000008,Ba1S3Te1,-4.37838,565.32959,20,icsd,0.424954,train,-4.370380
1,icsd_000012,Cl7Ga2K1,-3.34424,1132.18244,40,icsd,0.471884,train,-3.364022


In [15]:
df_pred['energy_err'] = (df_pred['energyperatom'] - df_pred['energy_predicted']).abs()
df_pred.groupby(['set', 'dataset']).energy_err.mean().unstack().round(3)

dataset,icsd,relax,vol
set,,,
test,0.045,0.034,0.050
test_composition,0.048,0.036,0.065
train,0.035,0.028,0.035
valid,0.052,0.038,0.048
